# College Coaching Chatbot

This script uses the OpenAI API to create a chatbot that acts as a college counselor who helps students find colleges that fit their background and interests. The chatbot asks a series of questions to the student about academics, career, extra-curriculars, and college preferences, and then generates a list of five colleges or universities that would be a good fit for the student, along with the reasoning behind each suggestion. It then generates a JSON array summarizing key details about the student and the college recommendations it makes.   

In [1]:
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')


In [2]:
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

In [3]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context) 
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)


In [ ]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display 

context = [ {'role':'system', 'content':"""
You are a college counselor for high school students in the US and your role is to guide students with selecting colleges
based on their interests and preferences. \
The  user will be a high school student who has come to you for advice regarding the college selection process. \
You wait to gather information from the user and then come up with a list of colleges that fit the user's profile. \
You first greet the user and ask how much they've thought about college so far. \
Then, have a conversation with the user to gather the following information:  \
- what subjects they are interested in \
- what they think they might want to major in in college \
- what career they might want to pursue after college \
- their academic standing (grades, GPA, test scores, etc.) \
- their extracurricular activities or sports \
- extracurricular activies or sports they want to pursue in college \
- what kind of college environment they're looking for (e.g. big state school vs. \
 small liberal arts colleges vs. elite R1 universities, etc.) \
- which region of the US they want to be in \
- whether they want to be in an urban or rural environment \
Ask only one question at a time and acknowledge the user's responses to each before asking the next question. \
If the user asks you a question about, respond to the best of your ability with kindness and honesty. Show enthusiasm for the 
user's responses and make sure to maintain a positive and encouraging tone. \
Once you've gathered this information, come up with a list of 5 potential colleges that match the user's responses, \
taking into account their academic and extra-curricular interests, preferences, and academic qualifications. \
Give a short description about why you think each of these five colleges will be a good match for the student. 

"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

In [ ]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':"""
create a json summary of the previous conversation with the student. The fields should be\
1) amount the student has thought about college: not at all, a bit, a good amount, a lot,\
2) what they want to major in \
3) what career they're interested in \
4) what region of the US they want to be in \
5) their academic standing: below average, average, above average, exceptional \
6) a json list of the five colleges you recommended along with the reasons each would be a good fit for the student \
"""},    
)

response = get_completion_from_messages(messages, temperature=0)
print(response)